In [2]:
import numpy as np
import pandas as pd
from Bio import SeqIO

## STRING NETWORK PROCESSING --> to Cytoscape

**Network assembly and identifier harmonization**

In this first part of the network analysis, a high-confidence protein–protein interaction (PPI) network for *Citrobacter rodentium* was constructed using data from STRING. Physical interaction links were imported and filtered by combined score (≥ 700) to retain only robust associations.

Protein identifiers were then harmonized by mapping STRING protein IDs (ROD-based) to NCBI RefSeq WP accessions using the STRING alias table. Duplicate edges and mirrored interactions (A–B / B–A) were removed, keeping only the highest-confidence edge per protein pair.

Finally, gene identifiers were standardized by prioritizing KEGG gene names when available and applying consistent fallbacks when missing. 

- The resulting network is ready for downstream subnetwork extraction, centrality analysis, and biological interpretation and can be opened in Cytoscape.


In [ ]:
# Input files downloaded from STRING for Citrobacter rodentium (TaxID: 637910)
file_int = "637910.protein.physical.links.full.v12.0.txt"
file_alias = "637910.protein.aliases.v12.0.txt"

# ================================
# 1. Load STRING alias information
# ================================
# This table maps STRING protein identifiers (e.g., 637910.ROD_xxxx)
# to alternative identifiers, including RefSeq WP protein IDs.
df_alias = pd.read_csv(file_alias, sep="\t", names=["string_protein_id", "alias", "source"])

# Keep only aliases corresponding to RefSeq protein accessions (WP_)
df_alias = df_alias[df_alias['alias'].str.startswith('WP_')]

# ================================
# 2. Load STRING physical interactions
# ================================
# The physical links file contains protein–protein associations
# inferred from experiments, databases, and transferred evidence.
df_int = pd.read_csv(file_int, sep=" ")

# Retain only high-confidence interactions
# combined_score > 700 corresponds to high confidence in STRING
df_int = df_int[df_int['combined_score'] > 700]

# Keep only relevant columns
df_int = df_int[['protein1', 'protein2', 'combined_score']]

# ==========================================
# 3. Convert STRING ROD IDs to RefSeq WP IDs
# ==========================================
# Build a mapping dictionary: {STRING_ID (ROD) -> WP_ID}
id_map = df_alias.set_index('string_protein_id')['alias'].to_dict()

# Replace protein identifiers in the interaction table
# If no WP mapping exists, keep the original identifier
df_int['protein1'] = df_int['protein1'].map(id_map).fillna(df_int['protein1'])
df_int['protein2'] = df_int['protein2'].map(id_map).fillna(df_int['protein2'])

# Remove exact duplicate rows, if any
df_int = df_int.drop_duplicates()

# ======================================================
# 4. Normalize interaction pairs and remove redundancies
# ======================================================
# STRING networks are undirected; therefore, interactions A–B and B–A
# should be treated as the same edge.
# Create a normalized pair representation (sorted tuple)
df_int["pair"] = df_int.apply(
    lambda x: tuple(sorted([x["protein1"], x["protein2"]])),
    axis=1)

# For each protein pair, keep only the interaction with the highest confidence score
df_clean = (
    df_int.sort_values("combined_score", ascending=False)
          .drop_duplicates(subset="pair", keep="first")
          .drop(columns="pair"))

# ================================
# 5. Export cleaned interaction network
# ================================
# The resulting file is a high-confidence, non-redundant
# protein–protein interaction network using WP IDs.
df_clean.to_csv("rede_crodentium_wp.net", sep="\t", index=False)

# Report network size before and after cleaning
print(f"Tamanho original: {len(df_int)}")       # original size
print(f"Tamanho após limpeza: {len(df_clean)}") # cleaned size

# export network
df_clean.to_csv("rede_crodentium_CS>700.net", sep="\t", index=False)

Tamanho original: 2072
Tamanho após limpeza: 1036


## Data Organization and Mapping to STRING Network

This second part of the network notebook focuses on data organization and identifier harmonization, integrating DEG-derived proteins with STRING aliases to prepare a consistent input for network construction and downstream topology analyses in Cytoscape.

In [ ]:
df_alias2 = pd.read_csv(file_alias, sep="\t", names=["string_protein_id", "alias", "source"])
df_alias2.head()


,string_protein_id,alias,source
0,#string_protein_id,alias,source
1,637910.ROD_00011,1.1.1.3,KEGG_EC
2,637910.ROD_00011,1.1.1.3,UniProt_DE_RecName_EC
3,637910.ROD_00011,2.7.2.4,KEGG_EC
4,637910.ROD_00011,2.7.2.4,UniProt_DE_RecName_EC


### STRING Alias table organization

#### Filtering WP ID in the aliases table

In [ ]:
def PivotAliasesBySource(
    df,
    sources_to_include,
    id_col='string_protein_id',
    alias_col='alias',
    source_col='source'
):
    """
    Pivot STRING alias table to wide format, generating one row per STRING protein
    and one column per selected alias source.

    PARAMETERS:
    df : pd.DataFrame --> STRING alias table (long format).
    sources_to_include : list[str] --> Alias sources to keep as columns.
    id_col : str --> Column with STRING protein identifiers.
    alias_col : str --> Column with alias values.
    source_col : str --> Column indicating alias source.

    RETURNS:
    pd.DataFrame --> Wide-format alias table with RefSeq-compatible WP IDs when available.
    """

    # Keep only selected alias sources
    df_filtered = df[df[source_col].isin(sources_to_include)].copy()

    # Special rule for RefSeq: keep only WP_ identifiers
    if 'UniProt_DR_RefSeq' in sources_to_include:
        mask_refseq = df_filtered[source_col] == 'UniProt_DR_RefSeq'
        df_filtered.loc[
            mask_refseq & ~df_filtered[alias_col].str.startswith('WP_'),
            alias_col
        ] = pd.NA

    # Pivot to wide format
    df_pivot = df_filtered.pivot_table(
        index=id_col,
        columns=source_col,
        values=alias_col,
        aggfunc='first'
    )

    # Ensure all requested sources exist
    for src in sources_to_include:
        if src not in df_pivot.columns:
            df_pivot[src] = pd.NA

    # Reorder columns
    df_pivot = df_pivot[sources_to_include]

    # Fallback: if no WP ID exists, keep STRING protein ID
    if 'UniProt_DR_RefSeq' in df_pivot.columns:
        df_pivot['UniProt_DR_RefSeq'] = df_pivot['UniProt_DR_RefSeq'].fillna(
            df_pivot.index.to_series())

    # Restore STRING protein ID as column
    df_pivot.reset_index(inplace=True)

    return df_pivot


In [7]:
sources = [
    'UniProt_DR_RefSeq',
    'KEGG_PRODUCT',
    'KEGG_NAME']

df_aliases = PivotAliasesBySource(df_alias2, sources_to_include=sources)
df_aliases

source,string_protein_id,UniProt_DR_RefSeq,KEGG_PRODUCT,KEGG_NAME
0,637910.ROD_00011,WP_012904386.1,NaN,ThrA
1,637910.ROD_00021,WP_012904387.1,Homoserine kinase,ThrB
2,637910.ROD_00031,WP_012904388.1,Threonine synthase,ThrC
3,637910.ROD_00041,WP_012904389.1,Conserved hypothetical protein,NaN
4,637910.ROD_00051,WP_012904390.1,"Alanine or glycine:cation symporter, AGCS family",NaN
...,...,...,...,...
4673,637910.ROD_50871,WP_012908898.1,Inner membrane protein,CreD
4674,637910.ROD_50881,WP_001194359.1,Aerobic respiration control protein,ArcA
4675,637910.ROD_50882,WP_001541509.1,Conserved hypothetical protein,YjjY
4676,637910.ROD_50891,637910.ROD_50891,Putative RNA methyltransferase,LasT


In [8]:
df_aliases.to_csv("alias_crodentium_wp_kegg.map", sep="\t", index=False)

#### Fill missing KEGG gene names using STRING protein identifiers.

In [ ]:
def preencher_nome(row):
    """
    Fill missing KEGG gene names using STRING protein identifiers.

    PARAMETERS:
    row : pd.Series --> Row from the alias DataFrame containing KEGG_NAME and string_protein_id.

    RETURNS:
    str --> KEGG gene name if available; otherwise, STRING protein ID without the taxon prefix.
    """

    # If KEGG_NAME is missing, derive a fallback name from the STRING protein ID
    if pd.isna(row["KEGG_NAME"]):
        return row["string_protein_id"].replace("637910.", "")
    else:
        return row["KEGG_NAME"]


# Apply the fallback rule row-wise to populate missing KEGG gene names
df_aliases["KEGG_NAME"] = df_aliases.apply(preencher_nome, axis=1)

# Inspect the first entries after name harmonization
df_aliases.head(10)


In [29]:
df_aliases.to_csv("alias_crodentium_wp_kegg.map", sep="\t", index=False)

### DEGs table organization

#### VF DEGs

In [ ]:
# Load DEGs annotated as virulence factors (VF)
# This table contains genes previously identified as VF-associated
# based on BLASTp against VFDB and DEG filtering.
df_vf = pd.read_csv("VF_DEGs_all.csv", sep="\t")
df_vf.head(5) 

,WP ID,Gene,log2FoldChange,padj,Annotation,VF_gene_name,VF_product,VFG ID,pident,length,evalue,bitscore,VFG_Annotation,Plot_names
0,WP_012907103.1,RS10235,4.810700,7.866238e-11,type III secretion system needle filament subu...,sctF,type III secretion system needle filament subu...,VFG041560(gb|WP_012907103),100.0,73,3.020000e-48,146.0,VFG041560(gb|WP_012907103) (sctF) type III sec...,RS10235 (sctF) type III secretion system needl...
1,WP_012907102.1,RS10240,4.670672,1.401668e-10,EscG/YscG/SsaH family type III secretion syste...,ROD_RS14650,EscG/YscG/SsaH family type III secretion syste...,VFG041562(gb|WP_012907102),100.0,98,9.050000e-69,200.0,VFG041562(gb|WP_012907102) (ROD_RS14650) EscG/...,RS10240 (ROD_RS14650) EscG/YscG/SsaH family ty...
2,WP_012907117.1,RS10155,4.661208,6.782228e-04,DUF1106 domain-containing protein,ROD_RS14735,DUF1106 domain-containing protein,VFG041548(gb|WP_012907117),100.0,138,6.850000e-100,282.0,VFG041548(gb|WP_012907117) (ROD_RS14735) DUF11...,RS10155 (ROD_RS14735) DUF1106 domain-containin...
3,WP_012907104.1,RS10230,4.396877,5.688094e-10,hypothetical protein,ROD_RS14660,hypothetical protein,VFG041561(gb|WP_012907104),100.0,135,9.680000e-100,281.0,VFG041561(gb|WP_012907104) (ROD_RS14660) hypot...,RS10230 (ROD_RS14660) hypothetical protein - E...
4,WP_012907105.1,RS10225,4.384187,7.899700e-11,attachment protein EaeB,ROD_RS14665,attachment protein EaeB,VFG041559(gb|WP_012907105),100.0,321,0.000000e+00,629.0,VFG041559(gb|WP_012907105) (ROD_RS14665) attac...,RS10225 (ROD_RS14665) attachment protein EaeB


#### Metabolic Pathways DEGs 

In [ ]:
# Load DEGs associated with metabolic pathways (PGs),
# obtained from KEGG-based pathway analysis (e.g. fructose, malate).
df_path = pd.read_csv("all_degs_pathways.csv", sep="\t")

# Harmonize KEGG-style ROD identifiers to STRING-compatible IDs
# This allows direct integration with the STRING PPI network
# (TaxID 637910 = Citrobacter rodentium).
df_path["ROD ID"] = df_path["ROD ID"].str.replace("^cro:", "637910.", regex=True)
df_path.head(5)

,metabolite,KEGG_pathway,WP ID,Gene,log2FoldChange,padj,Annotation,PG_gene_name,PG_product,ROD ID,pident,length,evalue,bitscore,PG_Annotation,Plot_names
0,Fructose,cro00010,WP_012905010.1,RS09065,1.599327,1.944054e-07,aldehyde dehydrogenase AldB,aldB,aldehyde dehydrogenase B (A),637910.ROD_42301,100.0,512,0.000000e+00,1065.0,cro:ROD_42301 K00138 aldehyde dehydrogenase [E...,RS09065 (aldB) aldehyde dehydrogenase B (A)
1,Fructose,cro00010,WP_012905011.1,RS06915,1.598626,2.774696e-04,hypothetical protein,pckA,phosphoenolpyruvate carboxykinase [ATP] (A),637910.ROD_44211,100.0,539,0.000000e+00,1118.0,cro:ROD_44211 K01610 phosphoenolpyruvate carbo...,RS06915 (pckA) phosphoenolpyruvate carboxykina...
2,Fructose,cro00010,WP_012905012.1,RS05440,1.424331,3.422083e-03,hypothetical protein,crr,glucose-specific PTS system EIIA component (A),637910.ROD_23761,50676.0,148,8.230000e-49,160.0,cro:ROD_23761 K02777 sugar PTS system EIIA com...,RS05440 (crr) glucose-specific PTS system EIIA...
3,Fructose,cro00010,WP_012905012.1,RS05615,1.140549,3.788610e-07,hypothetical protein,gpmA,"2,3-bisphosphoglycerate-dependent phosphoglyce...",637910.ROD_07471,100.0,250,0.000000e+00,513.0,"cro:ROD_07471 K01834 2,3-bisphosphoglycerate-d...","RS05615 (gpmA) 2,3-bisphosphoglycerate-depende..."
4,Fructose,cro00010,WP_012905076.1,RS11150,-1.122316,1.278176e-02,acetate--CoA ligase,acs,acetyl-coenzyme A synthetase (A),637910.ROD_34571,100.0,652,0.000000e+00,1353.0,cro:ROD_34571 K01895 acetyl-CoA synthetase [EC...,RS11150 (acs) acetyl-coenzyme A synthetase (A)


#### Mergings all DEGs

The resulting table contains all the information of DEGs including VF and metabolites identification.

It is prepared to be used as annotations in Cytoscape, in order to set up the genes in the network accordingly.

In [ ]:
# Merge VF-related DEGs and pathway-related DEGs into a single table
# This combined DEG set will be used as seed nodes for network extraction.
df_degs = pd.concat([df_vf, df_path], ignore_index=True)
df_degs.head(5)

,WP ID,Gene,log2FoldChange,padj,Annotation,VF_gene_name,VF_product,VFG ID,pident,length,evalue,bitscore,VFG_Annotation,Plot_names,metabolite,KEGG_pathway,PG_gene_name,PG_product,ROD ID,PG_Annotation
0,WP_012907103.1,RS10235,4.810700,7.866238e-11,type III secretion system needle filament subu...,sctF,type III secretion system needle filament subu...,VFG041560(gb|WP_012907103),100.0,73,3.020000e-48,146.0,VFG041560(gb|WP_012907103) (sctF) type III sec...,RS10235 (sctF) type III secretion system needl...,NaN,NaN,NaN,NaN,NaN,NaN
1,WP_012907102.1,RS10240,4.670672,1.401668e-10,EscG/YscG/SsaH family type III secretion syste...,ROD_RS14650,EscG/YscG/SsaH family type III secretion syste...,VFG041562(gb|WP_012907102),100.0,98,9.050000e-69,200.0,VFG041562(gb|WP_012907102) (ROD_RS14650) EscG/...,RS10240 (ROD_RS14650) EscG/YscG/SsaH family ty...,NaN,NaN,NaN,NaN,NaN,NaN
2,WP_012907117.1,RS10155,4.661208,6.782228e-04,DUF1106 domain-containing protein,ROD_RS14735,DUF1106 domain-containing protein,VFG041548(gb|WP_012907117),100.0,138,6.850000e-100,282.0,VFG041548(gb|WP_012907117) (ROD_RS14735) DUF11...,RS10155 (ROD_RS14735) DUF1106 domain-containin...,NaN,NaN,NaN,NaN,NaN,NaN
3,WP_012907104.1,RS10230,4.396877,5.688094e-10,hypothetical protein,ROD_RS14660,hypothetical protein,VFG041561(gb|WP_012907104),100.0,135,9.680000e-100,281.0,VFG041561(gb|WP_012907104) (ROD_RS14660) hypot...,RS10230 (ROD_RS14660) hypothetical protein - E...,NaN,NaN,NaN,NaN,NaN,NaN
4,WP_012907105.1,RS10225,4.384187,7.899700e-11,attachment protein EaeB,ROD_RS14665,attachment protein EaeB,VFG041559(gb|WP_012907105),100.0,321,0.000000e+00,629.0,VFG041559(gb|WP_012907105) (ROD_RS14665) attac...,RS10225 (ROD_RS14665) attachment protein EaeB,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_degs.to_csv("degs_VF_MP_crodentium.csv", sep="\t")